In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelBinarizer, FunctionTransformer, Normalizer
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime as dt
from sklearn import linear_model
# Create linear regression object
regr = linear_model.LinearRegression()
df = pd.read_csv('Dataset/melb_data.csv')
def label(column, raw=False):
    lbl = LabelBinarizer()
    if raw:
        lbl.fit(column)
        return lbl
    return pd.DataFrame(lbl.fit_transform(column), columns=lbl.classes_)
today = dt.now().date().toordinal() 
Date = pd.Series([today - dt.strptime(i, "%d/%m/%Y").date().toordinal() for i in df.Date])
Data = np.log(Date)
df.columns

In [ ]:
def outlier(column, m=1, plot = True):
    """My own function for detecting useful features"""
    Y = df.Price
    ind = []
    #return dir(Y)
    X = pd.concat([column,Y], axis='columns')
    X.dropna(inplace=True)
    Y = np.log(X[[Y.name]])
    X.drop(['Price'], axis='columns', inplace=True)
    regr.fit(X, Y)
    data = regr.coef_
    y=  data * [abs(data - np.mean(data)) > m * np.std(data)]
    if type(y[0]) == type(y):
        y = y[0][0]
    y = [ ind.append(i) or a if a else None for i, a in enumerate(y) ]
    if plot:
        plt.scatter(X.columns.map(str),y)
        plt.xticks(())
    return X.columns[ind]

In [ ]:
# df.Date = datter.fit_transform(df.Date)
types = label(df.Type)
method=label(df.Method)
suburb = label(df.Suburb)
df.Regionname.unique()
region = 1 * pd.DataFrame([('North' in i, 'West' in i, 'Metropolitan' in i) for i in df.Regionname], columns=['Northern', 'Western', 'Metropolitan'])
sellerG = label(df.SellerG)
seller = sellerG[outlier(sellerG, plot=False)]
suburbE = suburb[outlier(suburb, 7, plot = False)]

df.loc[:,('CouncilArea')] = df.CouncilArea.fillna('Unavailable')
c_area = label(df.CouncilArea)
post = label(df.Postcode)
postE = post[outlier(post, m=2, plot=True)]
all_sparse = pd.concat([sellerG, suburb], axis=1)
all_sparseE = all_sparse[outlier(all_sparse, m = 1, plot=False)]
all_sparseE.std()

In [ ]:
from sklearn.model_selection import train_test_split
X = pd.concat([df[
            ['Distance', 'Rooms', 'Car', 'Price', 'YearBuilt', 'Longtitude', 'Lattitude', 'Landsize',
            'BuildingArea', 'Propertycount', 'Bedroom2', 'Bathroom']],
                Date, types, method, c_area, post, region
              ], axis='columns')
X.Longtitude -= X.Longtitude.mean() 
X.Lattitude -= X.Lattitude.mean() 
X.YearBuilt = np.log(X.YearBuilt)
X.Bedroom2 **= 2
X.loc[:, ('Car')] = np.exp(X.loc[:, ('Car')])
X.loc[:, ('Rooms_2')] = X.loc[:, ('Rooms')]**2
X.loc[:,('Distance_2')] = X.loc[:,('Distance')]**2
X.dropna( inplace=True)
Y = np.log(X.Price)
X.drop(columns=['Price'], axis='columns',inplace=True)
X = X[outlier(X, 1e-9, plot=False)]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# Train the model using the training sets
res = regr.fit(X_train, Y_train)

# Plot coeffecients
plt.scatter(X.columns.map(str), regr.coef_)
plt.xticks(())
#plt.yticks(())
print('Accuracy : ', res.score(X_test,Y_test))
plt.show()
X.shape

In [ ]:
Coeffecients = zip(X.columns, res.coef_)
with open('model.csv', 'w') as fp:
    fp.write(
        'Features,Weight\n' + 
        '\n'.join(
            [
                print(i[1] == 0.0 and i[0]) or '"%s",%f' % i
                for i in Coeffecients
            ]
        )
    )

In [ ]:
for i in Coeffecients:
    print(i[1])